In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pickle
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Model



In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install pdpipe 
!pip install symspellpy
!pip install pycontractions

In [ ]:
import re
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import string
# !pip install symspellpy
import pkg_resources
from symspellpy.symspellpy import SymSpell
from symspellpy import SymSpell, Verbosity
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import csv
import pandas as pd

import nltk
nltk.download('punkt')

#Contraction Import
from pycontractions import Contractions

In [ ]:
train= pd.read_csv('../input/nlp-getting-started/train.csv')
test=pd.read_csv('../input/nlp-getting-started/test.csv')
train.head(3)

In [ ]:
from bs4 import BeautifulSoup
import requests, json, csv
import re

def remove_wrong_abb(key,value):
    if(re.search('it means',value)):
        return False;
    elif(re.search('\*',value)):
        return False
    elif(value==""):
        return False
    elif(len(key)>7):
        return False
    else :
        return True
    
resp = requests.get("http://www.netlingo.com/acronyms.php")
soup = BeautifulSoup(resp.text, "html.parser")
slangdict= {}
key=""
value=""
for div in soup.findAll('div', attrs={'class':'list_box3'}):
    for li in div.findAll('li'):
        for a in li.findAll('a'):
            key =a.text
            value = li.text.split(key)[1]
            if(remove_wrong_abb(key,value)):
                if(re.search('\ -or- ',value)):
                    pos=re.search('\ -or- ',value).start()
                    slangdict[key]=value[:pos]
                else:
                    slangdict[key]=value

                    
# with open('myslang.json', 'w') as f:
#     json.dump(slangdict, f, indent=2)

w = csv.writer(open("myslang.csv", "w"))
for key, val in slangdict.items():
    w.writerow([key.lower(), val.lower()])
    
reader = csv.reader(open('myslang.csv', 'r'))
abbreviations = {}
for row in reader:
    k, v = row
    abbreviations[k] = v

In [ ]:
sym_spell_4space = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell_4space.load_dictionary(dictionary_path, term_index=0, count_index=1)


sym_spell_misspelled = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
sym_spell_misspelled.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell_misspelled.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

In [ ]:
cont = Contractions(api_key="glove-twitter-100")
cont.load_models()

In [ ]:
#Version Edit: Sonam D.
def to_lower(text):
    text = text.lower()
    return text

#Version Edit: Anit G.
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+|pic.twitter.com\S+')
    return url.sub(r' ',text)

#Version Edit: Anit G.
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r' ',text)

#Version Edit: Anit G.
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

#Version Edit: Sonam D.
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

#Version Edit: Sonam D.
def remove_numbers(text):
    text = re.sub(r'\d+', ' ', text)
    return text

#Version Edit: Sonam D.
def remove_mentions(text):
    text = re.sub(r'@\w*', ' ', text)
    return text

#Version Edit: Sonam D.
def remove_punctuations(text):
    text = re.sub(r'([^A-Za-z \t])|(\w+:\/\/\S+)', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_square_bracket(text):
    text = re.sub('\[.*?\]', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_angular_bracket(text):
    text = re.sub('\<.*?\>+', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_newline(text):
    text = re.sub('\n', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_words_with_numbers(text):
    text = re.sub('\w*\d\w*', ' ', text)
    return text
    
#Version Edit: Susanth D.
def hashtag_to_words(text):
    text = re.sub(r'##', '#', text)
    hash_pattern = re.compile(r"#\w*")
    hashtag_list = re.findall(r"#\w+",text)
    for hashtag in hashtag_list:
        hashtag = re.sub(r'#', '', hashtag)
        text = re.sub(hashtag, sym_spell_4space.word_segmentation(hashtag).corrected_string, text)
    text = re.sub(r'#', '', text)
#     text = re.sub(r'# ', '', text)
    return text

#Version Edit: Susanth D.
def remove_stopwords(text):
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        if token not in stopwords.words('english'):
            textop = textop + token + ' '
    return textop

#Version Edit: Sonam D.
def correct_misspelled_with_context(text):
    suggestions = sym_spell_misspelled.lookup_compound(text, max_edit_distance=2)
    text = str(suggestions[0])
    text = re.sub(r', \d', ' ', text)
#     text = remove_numbers(text)
    return text

#Version Edit: Sonam D.
def stemming_text(text):
    stemmer= PorterStemmer()
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        textop = textop + stemmer.stem(token) + ' '
    return textop

#Version Edit: Sonam D.
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        textop = textop + lemmatizer.lemmatize(token) + ' '
    return textop

#Version Edit: Anit G.
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

#Version Edit: Anit G.
def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

#Version Edit: Sonam D.
# def handle_contractions(text):
#     text = re.sub(r"’", "'", text)
#     for word in text.split():
#         if word.lower() in contractions:
#             text = text.replace(word, contractions[word.lower()])
#     return(text)

#Version Edit: Saurabh M.
def removeRepeated(tweet):
    prev = ''
    tweet_new = ''
    for c in tweet:
        caps = False
        if c.isdigit():
            tweet_new += c
            continue
        if c.isalpha() == True:
            if ord(c) >= 65 and ord(c)<=90:
                caps = True
            c = c.lower()
            if c == prev:
                count += 1
            else:
                count = 1
                prev = c
            if count >= 3:
                continue
            if caps == True:
                tweet_new += c.upper()
            else:
                tweet_new += c
        else:
            tweet_new += c
    return tweet_new


def Expand_Contractions(text):
    return list(cont.expand_texts([text]))[0]

In [ ]:
pipeline = []
# pipeline.append(to_lower)
pipeline.append(remove_newline)
pipeline.append(remove_URL)
pipeline.append(remove_html)
pipeline.append(remove_emoji)
# pipeline.append(hashtag_to_words)
# pipeline.append(remove_words_with_numbers)
# pipeline.append(remove_numbers)
pipeline.append(remove_mentions)
pipeline.append(remove_square_bracket)
pipeline.append(remove_angular_bracket)
pipeline.append(Expand_Contractions)
# pipeline.append(remove_punctuations)
pipeline.append(removeRepeated)
# pipeline.append(convert_abbrev_in_text)
# pipeline.append(remove_stopwords)
# pipeline.append(correct_misspelled_with_context)
# pipeline.append(remove_numbers)
# pipeline.append(remove_stopwords)
# # pipeline.append(stemming_text)
pipeline.append(lemmatization)

train = preprocess_pipeline(pipeline, 'text', train)
test = preprocess_pipeline(pipeline, 'text', test)

In [ ]:
CRAWL_EMBEDDING_PATH = '../input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl'
GLOVE_EMBEDDING_PATH = '../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'

In [ ]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [ ]:
sub = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')
sub.head()

In [ ]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
train.head()

In [ ]:
TEXT_COLUMN = 'text'
TARGET_COLUMN = 'target'
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'

In [ ]:
x_train = train[TEXT_COLUMN].astype(str)
y_train = train[TARGET_COLUMN].values
x_test = test[TEXT_COLUMN].astype(str)

In [ ]:
tokenizer = text.Tokenizer(filters=CHARS_TO_REMOVE, lower=False)
tokenizer.fit_on_texts(list(x_train) + list(x_test))

In [ ]:
crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): ', len(unknown_words_crawl))

glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

In [ ]:
embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
embedding_matrix.shape

In [ ]:
MAX_LEN = 300
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS

In [ ]:
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN)

In [ ]:
def build_model(embedding_matrix):
    words = Input(shape=(None,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    
    model = Model(inputs=words, outputs=result)
    model.compile(loss='binary_crossentropy', optimizer='adam')

    return model

In [ ]:
NUM_MODELS = 2
BATCH_SIZE = 512
EPOCHS = 5

In [ ]:
checkpoint_predictions = []
weights = []

for model_idx in range(NUM_MODELS):
    model = build_model(embedding_matrix)
    for global_epoch in range(EPOCHS):
        model.fit(
            x_train,y_train,
            batch_size=BATCH_SIZE,
            epochs=1,
            verbose=2
        )
        checkpoint_predictions.append(model.predict(x_test, batch_size=BATCH_SIZE))
        weights.append(2 ** global_epoch)

In [ ]:
predictions = np.average(checkpoint_predictions, weights=weights, axis=0)

In [ ]:
predictions.shape

In [ ]:
sub.iloc[:, 1] = (predictions > 0.5).astype(int)

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
from collections import Counter
Counter(sub['target'])